In [1]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('..')

import numpy as np
import torch

from pytorch_pretrained_bert.tokenization import BertTokenizer

from lib.bert import BertForSequenceClassification
from lib import size_utils, tasks, data_processors, feature_processors, metrics

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
%env CUDA_VISIBLE_DEVICES=0

params = {
    'data_dir': '../../data/SST-2',
    'cache_dir': '../model_cache',
    'task_name': 'sst2',
    'bert_model': 'bert-base-uncased',
    'max_seq_length': 128,
    'eval_batch_size': 20,
    'device': torch.device(
        'cuda' if torch.cuda.is_available()
        else 'cpu')
}

params['num_labels'] = tasks.num_labels[params['task_name']]
params['label_list'] = tasks.label_lists[params['task_name']]

env: CUDA_VISIBLE_DEVICES=0


In [3]:
processor = tasks.processors[params['task_name']]()
tokenizer = BertTokenizer.from_pretrained(
    params['bert_model'], do_lower_case=True)

# We take only one batch of evaluation examples.
eval_examples = processor.get_dev_examples(
    params['data_dir'])[:params['eval_batch_size']]

model = BertForSequenceClassification.from_pretrained(
    params['bert_model'],
    cache_dir=params['cache_dir'],
    num_labels=params['num_labels']).to(params['device'])

### Counting the number of weights

In [4]:
num_params = {
    'embedding': 0,
    'encoder': 0,
    'pooler': 0,
    'total': 0
}

weights = model.state_dict()
for layer_name, layer in weights.items():
    layer_params_num = len(layer.cpu().numpy().ravel())
    for name in num_params:
        if name in layer_name:
            num_params[name] += layer_params_num
    num_params['total'] += layer_params_num
    
num_params

{'embedding': 23837184,
 'encoder': 85054464,
 'pooler': 590592,
 'total': 109483778}

In [5]:
num_params_shares = {
    layer: num_params[layer] * 1.0 / num_params['total']
    for layer in num_params
    if layer != 'total'
}

num_params_shares

{'embedding': 0.21772343296374008,
 'encoder': 0.7768681858969098,
 'pooler': 0.005394333396131069}

### Checking model size

In [6]:
size_utils.get_model_size(model, params['cache_dir'])

'417.69311809539795 MB'

### Applying to a single batch

In [7]:
eval_features = feature_processors.convert_examples_to_features(
    eval_examples,
    params['label_list'],
    params['max_seq_length'],
    tokenizer)
input_ids = torch.tensor(
    [f.input_ids for f in eval_features],
     dtype=torch.long).to(params['device'])
input_mask = torch.tensor(
    [f.input_mask for f in eval_features],
     dtype=torch.long).to(params['device'])
segment_ids = torch.tensor(
    [f.segment_ids for f in eval_features],
     dtype=torch.long).to(params['device'])

In [17]:
model.eval()
logits = model(input_ids, segment_ids, input_mask).detach().cpu()
softmax = torch.nn.Softmax(dim=-1)
y_pred = softmax(logits).numpy()
y_true = np.array([int(example.label) 
                   for i, example in enumerate(eval_examples)])
y_pred

array([[0.5950677 , 0.4049324 ],
       [0.633476  , 0.36652398],
       [0.62468827, 0.3753117 ],
       [0.5904694 , 0.40953058],
       [0.6217312 , 0.37826872],
       [0.5833395 , 0.4166605 ],
       [0.61120874, 0.38879126],
       [0.57841194, 0.42158806],
       [0.61265224, 0.38734776],
       [0.6302714 , 0.36972862],
       [0.5941279 , 0.40587205],
       [0.599353  , 0.40064695],
       [0.59727263, 0.40272734],
       [0.59377027, 0.4062298 ],
       [0.60265714, 0.3973429 ],
       [0.5771439 , 0.42285612],
       [0.56967974, 0.43032026],
       [0.5727943 , 0.42720568],
       [0.602853  , 0.39714694],
       [0.6229316 , 0.3770684 ]], dtype=float32)

Какая точность у незафайнтьюненной модели?

In [24]:
metrics.accuracy(y_true, y_pred)

0.5